# Autoencoding for analysis of spectroscopic data

---

Lecture: "Physics-augmented machine learning" @ Cyber-Physical Simulation, TU Darmstadt

Lecturer: Prof. Oliver Weeger

Assistants: Dr.-Ing. Maximilian Kannapin, Jasper O. Schommartz, Dominik K. Klein

Summer term 2025

---



*Run the following cell to clone the GitHub repository in your current Google Colab environment.*

In [3]:
!git clone https://github.com/CPShub/LecturePhysicsAwareML.git
! git clone https://github.com/csho33/bacteria-ID.git

fatal: destination path 'LecturePhysicsAwareML' already exists and is not an empty directory.
Cloning into 'bacteria-ID'...
remote: Enumerating objects: 32, done.
remote: Total 32 (delta 0), reused 0 (delta 0), pack-reused 32 (from 1)
Receiving objects: 100% (32/32), 14.37 MiB | 30.71 MiB/s, done.
Resolving deltas: 100% (9/9), done.


*Run the following cell to import all modules and python files to this notebook. If you made changes in the python files, run the following cell again to update the python files in this notebook. You might need to restart your Colab session first ("Runtime / Restart session" in the header menu).*


In [4]:
import tensorflow as tf
import datetime
now = datetime.datetime.now
import LecturePhysicsAwareML.Autoencoder.data as ld
import LecturePhysicsAwareML.Autoencoder.models as lm
import LecturePhysicsAwareML.Autoencoder.plots as lp

ModuleNotFoundError: No module named 'LecturePhysicsAwareML.Autoencoder.data'

*Run this cell if you are executing the notebook locally on your device.*

In [ ]:
import tensorflow as tf
import datetime
now = datetime.datetime.now
import data as ld
import models as lm
import plots as lp

Matplotlib is building the font cache; this may take a moment.


*If you want to clone the repository again, you have to delete it from your Google Colab files first. For this, you can run the following cell.*

In [ ]:
%rm -rf LecturePhysicsAwareML

In [ ]:
cases = [18,27,0,26]
#cases = [18,27,0]
#cases = [18,27]
#cases=[0,26]
wn_c, wn_t, wn_v, spectra_c, spectra_t, spectra_v, label_c, label_t, label_v = hd.load_data(cases,int(70*len(cases)),int(15*len(cases)))

# %%
"""
Initialise NNs

"""

latent_variables = 4
nodes = 64

units = [nodes,latent_variables,nodes,1000]
activation = ['softplus','linear','softplus','linear']
model_AE = hm.main(units=units, activation=activation)

units = [nodes,latent_variables]
activation = ['softplus','linear']
model_E = hm.main(units=units, activation=activation)



# %%
"""
Calibrate autoencoder

"""

epochs = 500
h = model_AE.fit([spectra_c], [spectra_c], epochs=epochs, verbose=2, validation_data=(spectra_v,spectra_v))

hp.plot_loss(h)


# %%
"""
Visualise latent space

"""

model_E.set_weights(model_AE.weights[0:4])

for i in range(latent_variables):
    for j in range(latent_variables):
        if i!=j:
            if i>j:

                hp.plot_latent_space_ij(model_E, spectra_c, spectra_t, label_c, label_t, i, j)


# %%
"""
Visualise bacteria

"""

for i in range(len(cases)):

    wn, spectra = hd.load_single_case(cases[i])
    hp.plot_spectra(wn, spectra, i, cases[i])